<a href="https://colab.research.google.com/github/Chris1Areiza/lime/blob/main/AB_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Resolución del AB Challenge**

##**Librerías y carga de datos**

In [4]:
# Importar las librerías necesarias.
import pandas as pd
from google.colab import drive
from datetime import datetime
import numpy as np
from scipy.stats import f_oneway, ttest_ind
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from google.colab import auth
from google.cloud import bigquery
import tempfile

In [5]:
#Leer los datos desde el csv file.
drive.mount('/content/drive')
data=pd.read_csv('drive/MyDrive/datameli.csv',sep=',', index_col = None)

Mounted at /content/drive


In [6]:
# Imprimir la forma y los primeros registros del dataset.
print(data.shape)
print(data.head())

(141553, 6)
  event_name      item_id                     timestamp site  \
0     SEARCH          NaN  2021-08-02T23:55:38.966-0400  MLA   
1    PRODUCT  882352139.0  2021-08-02T23:55:51.673-0400  MLA   
2    PRODUCT  655266729.0  2021-08-02T23:56:16.083-0400  MLA   
3    PRODUCT  761520929.0  2021-08-02T23:56:29.989-0400  MLA   
4    PRODUCT  757586409.0  2021-08-02T23:56:47.887-0400  MLA   

                                         experiments  user_id  
0  {searchbackend/recommended-products=6157, mcli...  3204901  
1  {qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...  3204901  
2  {qadb/sa-on-vip=6695, vip/showV2V3BoxMessages=...  3204901  
3  {search/remove-ecn-tag=4954, qadb/sa-on-vip=66...  3204901  
4  {search/remove-ecn-tag=4954, qadb/sa-on-vip=66...  3204901  


El dataset contiene 141553 registros y 6 columnas como se explicó en el enunciado.

In [10]:
# Contar entradas que continen el valor BUY.
count_buy = len(data[data['event_name'] == 'BUY'])
print("Número de valores 'BUY':", count_buy)

Número de valores 'BUY': 1088


Se observa que de los 141553 registros solo 1088 corresponden a compra realizada, es decir, el 0.77% del total de registros.

In [8]:
# Obtener información de los datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141553 entries, 0 to 141552
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   event_name   141553 non-null  object 
 1   item_id      64803 non-null   float64
 2   timestamp    141553 non-null  object 
 3   site         141553 non-null  object 
 4   experiments  141553 non-null  object 
 5   user_id      141553 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 6.5+ MB


Todas las columnas tienen 141553 valores excepto item_id que solo contiene 64803 entradas no nulas, sin embargo, no se considera relevante ya que el análisis se enfoca a los experimentos no al item como tal.

##**Procesamiento para agregación**

In [11]:
# Convertir la columna timestamp a tipo datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Crear una columna Day con el formato YYYY-MM-DD
data['Day'] = data['timestamp'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d %H'))

# Dividir la columna experiments en dos nuevas columnas: Experiment y Variant
data[['Experiment', 'Variant']] = data['experiments'].str.extractall(r'([^=,}]+)=([^=,}]+)').reset_index(drop=True)[[0, 1]]

# Contar la cantidad de user_id y BUY para cada combinación de Day, Experiment y Variant
agg_data = data.groupby(['Day', 'Experiment', 'Variant']).agg({'user_id': 'nunique'}).reset_index()
buy_data = data[data['event_name'] == 'BUY'].groupby(['Day', 'Experiment', 'Variant']).agg({'event_name': 'count'}).reset_index()

# Unir los dos dataframes en uno
final_data = pd.merge(agg_data, buy_data, on=['Day', 'Experiment', 'Variant'], how='outer').fillna(0)
final_data.columns = ['Day', 'Experiment', 'Variant', 'Users', 'Buy']

# Ordenar el dataframe por Day, Experiment y Variant
final_data = final_data.sort_values(['Day', 'Experiment', 'Variant'])


En el procesamiento para agregación se busca transformar la cadena de navegación de los usuario en la muestra a una agregación más útil en la que se puede ver a nivel de hora y día (Day) el resultado de cada experimento en cada una de sus variantes, el resultado se puede evaluar como cantidad de usuarios que participaron del experimento y cantidad de compras asociadas a la variante.

In [12]:
# Imprimir el dataframe final
final_data

,Day,Experiment,Variant,Users,Buy
0,2021-08-01 09,cookiesConsentBanner,DEFAULT,1,0.0
1,2021-08-01 09,frontend/assetsCdnDomainMLA,DEFAULT,1,0.0
2,2021-08-01 09,mclics/show-pads-global,5176,1,0.0
3,2021-08-01 09,mclics/show-pads-search-list,5146,1,0.0
4,2021-08-01 09,pdp/viewItemPageMigrationDesktopReviewsNoTabs,4856,1,0.0
...,...,...,...,...,...
1847,2021-08-02 23,{search/results-target-web-motors,7327,1,0.0
1848,2021-08-02 23,{search/results-target-web-motors,7328,5,0.0
1849,2021-08-02 23,{searchbackend/recommended-products,6157,28,0.0
1850,2021-08-02 23,{searchbackend/recommended-products,6158,38,2.0


In [13]:
# Calcular la cantidad de variantes por experimento
variants_per_experiment = final_data.groupby('Experiment')['Variant'].nunique()
print(variants_per_experiment)


Experiment
 buyingflow/address_hub                               1
 buyingflow/escWebMLA                                 1
 buyingflow/secure_card                               1
 cookiesConsentBanner                                 1
 filters/sort-by-ranking                              3
 frontend/assetsCdnDomainMLA                          1
 frontend/assetsCdnDomainMLU                          1
 mclics/ads-adsearch-boost-incremental-desktop-mla    1
 mclics/search-list-algorithms                        1
 mclics/search-pads-none-desktop-mla                  2
 mclics/show-pads-global                              2
 mclics/show-pads-search-list                         2
 mshops/HideTransitionModal                           4
 pdp/cpgShowOnlyAddToCart                             1
 pdp/viewItemPageMigrationDesktopHirableSRV           1
 pdp/viewItemPageMigrationDesktopQuotableSRV          1
 pdp/viewItemPageMigrationDesktopReviewsNoTabs        2
 pdp/viewItemPageMigrationReturns    

Con el fin de conocer un poco más el comportamiento de los experimentos, se observa que 25 (53.2%) tienen una sola variante asociada, 19 (40.4%) tienen dos variantes asociadas, 2 (4.3%) tienen tres variantes asociadas y 1 (2.1%) tiene cuatro variantes asociadas.

##**Función para el AB Test por experimento**

In [14]:
def ab_testing(data):
    unique_variants = data['Variant'].nunique()

    # Condición 1: Una variante única
    if unique_variants == 1:
        return data['Variant'].iloc[0]

    # Condición 2: Dos variantes únicas
    elif unique_variants == 2:
        variant1_data = data[data['Variant'] == data['Variant'].unique()[0]]['Buy']
        variant2_data = data[data['Variant'] == data['Variant'].unique()[1]]['Buy']
        t_stat, p_val = ttest_ind(variant1_data, variant2_data)

        if p_val < 0.05:
            return data.groupby('Variant')['Buy'].mean().idxmax()
        else:
            return 'None'

    # Condición 3: Tres o cuatro variantes únicas
    elif unique_variants in [3, 4]:
        anova_data = [data[data['Variant'] == v]['Buy'].tolist() for v in data['Variant'].unique()]
        f_stat, p_val = f_oneway(*anova_data)

        if p_val < 0.05:
            mc = pairwise_tukeyhsd(np.concatenate(anova_data), np.repeat(data['Variant'].unique(), [len(a) for a in anova_data]), alpha=0.05)
            winner_variant = None

            for i, v in enumerate(data['Variant'].unique()):
                if mc.reject[mc.groupsunique.tolist().index(v)]:
                    if winner_variant is None or data[data['Variant'] == v]['Buy'].mean() > data[data['Variant'] == winner_variant]['Buy'].mean():
                        winner_variant = v
            return winner_variant
        else:
            return 'None'

    else:
        raise ValueError("Número de variantes no soportado")

# Aplicar la función ab_testing a cada grupo de experimentos
winner_variants = final_data.groupby(['Experiment']).apply(ab_testing).reset_index()
winner_variants.columns = ['Experiment', 'winner_variant']

# Agregar la columna 'winner_variant' al dataframe 'final_data'
final_data = final_data.merge(winner_variants, on='Experiment', how='left')



/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:3895: ConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(stats.ConstantInputWarning(msg))
/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.9/dist-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


Con el fin de encontrar las variantes ganadoras (winner_variant) por cada experimento se realizo un AB Test con diferentes condiciones.
- Si un experimento posee una sola variante única entonces la variante ganadora será la misma variante ya que no se tiene referencia para realizar una comparación mediante test estadísticos.
- Si un experimento posee dos variantes únicas entonces se realiza una prueba t para dos muestras independientes para comparar el promedio de las compras en ambas variantes. Si el valor-p es menor al nivel de significancia por defecto de 0.05 entonces existe una diferencia significativa entre las medias, es decir, se rechaza Ho y la función devolverá la variante con la media más alta. En caso de que no exista diferencia significativa la función devolverá 'None' ya que ninguna de las variantes es la ganadora, por lo tanto sería indiferente escoger una u otra.
- Si un experimento posee tres o más variantes únicas se realiza un test de ANOVA para comparar las medias de las compras en las diferentes variantes. Si el valor-p es menor a 0.05 entonces existe diferencia significativa en al menos dos de las medias, luego se realiza una prueba port-hoc conocida como test de Tukey para identificar cual de las variantes posee una media significativamente más alta que las demás. Al final la función devuelve la variante ganadora. En caso de que no hayan diferencias signifitivas entre las diferentes medias la función devuelve 'None' ya que sería indiferente escoger una u otra variante pues todas generarían el mismo nivel de compras. 

Si las variantes unicas no son 1,2 3 o 4 entonces se genera un error indicando que la cantidad de variantes no es compatible. Aunque probar muchos grupos a la vez puede ser contraproducente para el experimento.

Se aplica la función a cada grupo de experimentos en el dataframe y el resultado se almacena en el dataframe winner_variants y luego se combina con el dataframe final_data mediante merge en función de la columna 'Experiment' para agregar la nueva columna 'winner_variant' al dataframe final_data.

Las asunciones y supuestos estadísticos, además del diseño de experimentos se puntualiza en el Read.me del repositorio.

In [15]:
# Imprimir el dataframe final
final_data


,Day,Experiment,Variant,Users,Buy,winner_variant
0,2021-08-01 09,cookiesConsentBanner,DEFAULT,1,0.0,DEFAULT
1,2021-08-01 09,frontend/assetsCdnDomainMLA,DEFAULT,1,0.0,DEFAULT
2,2021-08-01 09,mclics/show-pads-global,5176,1,0.0,5176
3,2021-08-01 09,mclics/show-pads-search-list,5146,1,0.0,5146
4,2021-08-01 09,pdp/viewItemPageMigrationDesktopReviewsNoTabs,4856,1,0.0,None
...,...,...,...,...,...,...
1847,2021-08-02 23,{search/results-target-web-motors,7327,1,0.0,None
1848,2021-08-02 23,{search/results-target-web-motors,7328,5,0.0,None
1849,2021-08-02 23,{searchbackend/recommended-products,6157,28,0.0,None
1850,2021-08-02 23,{searchbackend/recommended-products,6158,38,2.0,None


In [16]:
# Guardar el dataframe final en Drive como .txt
file_path = '/content/drive/MyDrive/final_data.txt'
final_data.to_csv(file_path, sep='\t', index=False)


Se guarda el dataframe final_data en un archivo .txt que posteriormente será cargado a github para que la API lo consuma.

##**Cargue de la data a BigQuery**

In [ ]:
# Instala la biblioteca de cliente de BigQuery para Python.
!pip install google-cloud-bigquery


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Autenticar mi cuenta de Google en Colab.
auth.authenticate_user()

In [ ]:
# Definir las variables del proyecto.
project_id = 'prueba-meli-384022'
dataset_id = 'abtesting'
table_id = 'final_data'


In [ ]:
# Crear una instancia del cliente de BigQuery y un objeto TableReference para la tabla de destino
client = bigquery.Client(project=project_id)
table_ref = client.dataset(dataset_id).table(table_id)

In [ ]:
# Crear el conjunto de datos si aún no existe.
dataset = bigquery.Dataset(client.dataset(dataset_id))
try:
    client.create_dataset(dataset)
    print(f"Dataset {dataset_id} creado.")
except Exception as e:
    print(f"El dataset {dataset_id} ya existe.")

El dataset abtesting ya existe.


In [ ]:
# Guardar el DataFrame como un archivo Parquet temporal
temp_parquet = tempfile.NamedTemporaryFile(delete=False, suffix='.parquet')
final_data.to_parquet(temp_parquet.name)

# Cargar el archivo Parquet temporal en BigQuery
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.PARQUET
job_config.autodetect = True

with open(temp_parquet.name, "rb") as source_file:
    job = client.load_table_from_file(source_file, 'prueba-meli-384022.abtesting.final_data', job_config=job_config)
job.result()

# Eliminar el archivo Parquet temporal
temp_parquet.close()


De esta manera se carga el dataframe final_data en una tabla en un dataset en un proyecto en BigQuery. La intención es disponer la tabla en la nube de Google para que pueda ser consumida por algún otro servicio. Inicialmente se dispuso en BigQuery para que la API la consumiera, por practicidad se decidió finalmente que la API consumiera la data desde un txt en github pero la tabla quedó cargada en BigQuery. La evidencia se adjunta en el read.me del repositorio.